<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/1_mice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import mf
from google.colab import drive
drive.mount('/content/drive')

for dirname, _, filenames in os.walk('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ModuleNotFoundError: ignored

# 1. Import dataset e descrizione feature


* TransactionDT: timedelta from a given reference datetime

* TransactionAMT: transaction payment amount in USD

* card4: mastercard, visa and other.

* card6: credit or debit.

* P_ and (R__) emaildomain: purchaser and recipient email domain. Categorical: gmail.com, hotmail.com and others.

ProductCD: product code, the product for each transaction. Categorical. 

C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.

D1-D15: timedelta, such as days between previous transaction, etc.

M1-M9: match, such as names on card and address, etc. Categorical: T or F.  

Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

* DeviceType: type of device used, mobile or desktop.

* DeviceInfo: info on device.

* id_30: OS name and version.

* id_31: browser name and version.

* id_33: screen size.

id_35 to 38: T or F.

id_34: match_status:1 or 2.
 
id_15 and 28: New or Found. Maybe Found if a customer is registered in Vesta database. 

id_12 and 16: Found or NotFound. 

id_27: Null of Found.

Vado ad importare il training set

Unisco i due dataset tramite la funzione `merge`, che li unisce rispetto alla colonna `TransactionID`. 

`original_data` conterrà il dataset originale, senza rimpiazzamento dei missing values nè encoding delle features categoriche. 

Funzione `merge`: https://stackoverflow.com/questions/41463119/join-two-dataframes-on-common-column-in-python

`to_csv`: https://datatofish.com/export-dataframe-to-csv/

In [ ]:
if not os.path.isfile('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/original_data.csv'):
    data_identity = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/train_identity.csv')
    data_transaction = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/train_transaction.csv')
    # unisco i due dataframe
    data = pd.merge(data_transaction, data_identity, left_on='TransactionID', right_on='TransactionID', how='left')
    original_data = data

    del data
    del data_transaction
    del data_identity

    original_data.to_csv(r'./original_data.csv')

original_data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/original_data.csv')
original_data.drop(original_data.columns[original_data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

## Missing values: MICE method

Seleziono i NaN dal dataframe e li conto. 

Costruisco `nans`, che è un dataframe che contiene (feature, nan_totali). 

https://stackoverflow.com/questions/28503445/assigning-column-names-to-a-pandas-series

Plotto i NaN contati per ogni feature. 

In [ ]:
nan_values = original_data.isna().sum()
nans = pd.DataFrame(nan_values).reset_index()
nans.columns = ['feature', 'count']
nans = nans[nans['count'] != 0]
del nan_values

do = False
if do:
    i = 0
    while i < 404:
        sns.catplot(y ='feature', x="count", kind="bar", original_data=nans[i:i + 30])
        i += 30
    # ax.set(xscale="log")
    nans_sorted = nans['count'].sort_values()
    plt.figure(figsize=(20,10))
    nans_sorted.plot(kind = 'bar')
    plt.show()

In [ ]:
del nans

### MICE: Multivariate Imputation by Chained Equations

Fa parte dei metodi a multiple imputation, ovvero quelli in cui i missing values sono generati più volte dal dataset. I dataset ottenuti vengono poi uniti, e i dati che rimpiazzeranno i missing values vengono scelti tramite qualche regola. 

### Fasi dei metodi a multiple imputation:
1. imputation: calcolo in qualche modo il valore da assegnare al missing value;
2. analisi dei risultati ottenuti dalle varie imputation;
3. pooling: integrazione dei risultati nel dataset finale.

### Fasi del metodo MICE:
1. faccio una simple imputation per ogni missing value nel dataset;
2. i missing value sono ripristinati;
3. i valori osservati, cioè quelli generati dalle simple imputations, sono regressed dalle altre variabili nell'imputation model. 
4. i missing values sono quindi rimpiazzati con i valori predetti dal modello di regressione;
5. gli step dal 2 al 4 vengono ripetuti per ogni variabile che ha missing values. La ripetizione di questi step costisuisce un ciclo. Alla fine di un ciclo i missing values sono stati rimpiazzati dai valori predetti dalla regressione che riflette le relazioni osservate nel dati;
6. gli step dal 2 al 4 sono ripetuti per un numero fissato di cicli, e ad ogni ciclo i valori predetti vengono uppati. Alla fine di questi cicli i valori finali predetti andranno nel dataset. 

Di solito si effettuano dieci cicli. 
Idea: alla fine dei cicli, la distribuzione dei parametri che fornisce i valori predetti, deve convergere e diventare stabile. 

**Descrizione da scikit-learn:** models each feature with missing values as a function of other features, and uses that estimate for imputation. It does so in an iterated round-robin fashion: at each step, a feature column is designated as output y and the other feature columns are treated as inputs X. A regressor is fit on (X, y) for known y. Then, the regressor is used to predict the missing values of y. This is done for each feature in an iterative fashion, and then is repeated for max_iter imputation rounds. The results of the final imputation round are returned.

Riferimenti: https://www.kaggle.com/residentmario/simple-techniques-for-missing-data-imputation

IterativeImputer: https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html

Uso IterativeImputer: https://www.geeksforgeeks.org/missing-data-imputation-with-fancyimpute/

Note: MICE era prima presente in fancyimpute, ma poi è stato spostato in scikit-learn

`split_by_day`: Funzione che splitta il `dataset` nel numero di `days` specificato. 

`select_col_by_nan`: seleziona le colonne del `dataset` se il numero di NaN che contengono è inferiore alla soglia `tresh`. 

`mice`: performa il MICE sul `dataset` specificato, considerando il numero di `days` specificato. 

Faccio l'IterativeImputer su un singolo giorno. -> L'algoritmo non riesce a raggiungere la convergenza se ci sono colonne con troppi NaN.
Errore:

`/opt/conda/lib/python3.7/site-packages/sklearn/impute/_iterative.py:670: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)`
  
Per risolvere il problema seleziono le colonne con meno di 100k NaN e aumento max_iter a 40. Paper [M.A] sottolineava come fossero troppo ottimistiche le stime di max_iter 10 per casi con tanti missing values. 

Dopo che ho fatto l'imputing, il dataset viene riunito nuovamente e viene prodotto un csv. 

Riferimento paper [M.A]: https://drive.google.com/file/d/1aK5py9lvjo6WHFDIROS7frXG-f-O5TEN/view?usp=sharing

In [ ]:
if not os.path.isfile('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/fitted.csv'):
# if os.path.isfile('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/fitted.csv'):
    numerical_data = original_data.select_dtypes(include=['int64','float64'])
    fitted = mice(numerical_data, days=182)
    selected_cols = select_col_by_nan(numerical_data, tresh=100000)
    print('Number of selected columns: ', len(selected_cols))
    fitted1 = pd.concat(fitted)
    fitted1
    fitted1.to_csv(r'./fitted.csv')

numerical_data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/fitted.csv')
numerical_data.drop(numerical_data.columns[numerical_data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

## Dataset split in numerical e categorical

Seleziono le feature i cui dati sono numerici. 

https://pandas.pydata.org/pandas--docs/stable/reference/api/pandas.DataFrame.select_dtypes.html

In [ ]:
num_data = numerical_data.drop(['isFraud','TransactionID','TransactionDT'], axis = 1)

In [ ]:
categorical_col = []
categorical_col.append('TransactionID')
categorical_col.append('isFraud')
for col in original_data:
  if col not in numerical_cols:
    categorical_col.append(col)

len(categorical_col)
categorical_data = original_data[original_data.columns.intersection(categorical_col)]